In [46]:
import os
import webp
import shutil
from PIL import Image
import numpy as np
from diffusers import DiffusionPipeline, StableDiffusionImg2ImgPipeline
from transformers import pipeline
from pytorch_gan_metrics import get_inception_score
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import subprocess

torch.cuda.set_device(1)

In [14]:
def clear_dir(path):
    for i in os.listdir(path):
        new_path = path + '/' + i
        if len(i.split('.')) > 1 and i.split('.')[-1] != i.split('.')[-1].upper() and '_' not in i.split('.')[-1]:
            os.remove(new_path)
        else:
            try:
                clear_dir(new_path)
                os.rmdir(new_path)
            except NotADirectoryError:
                os.remove(new_path)

In [52]:
packs_list = ['adrianochelentano', 'animals', 'daenerys', 'alicefox', 'bender']
user_images_path = "user_images_data"
gen_images_path = "gen_images_data"
pack_images_path = "train_data/downloaded/"
base_model_path = "tuned_model256"
model_id = "onpack_tuned_model"
user_images_base = "user_base"

final_style_sum = 0
final_init_sum = 0
#3
for pack in packs_list:
    for img in os.listdir(user_images_base):
        shutil.copy2(user_images_base + "/" + img, user_images_path)
 
    pack_images_path = pack_images_path + pack
    
    !accelerate launch train_model.py \
      --pretrained_model_name_or_path="tuned_model256"  \
      --instance_data_dir="train_data/downloaded/adrianochelentano" \
      --output_dir="onpack_tuned_model" \
      --instance_prompt="a cartoon sticker" \
      --resolution=256 \
      --train_batch_size=1 \
      --gradient_accumulation_steps=1 \
      --learning_rate=5e-6 \
      --lr_scheduler="constant" \
      --lr_warmup_steps=0 \
      --max_train_steps=400

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float32, safety_checker = None).to("cuda")

    #1 и 2
    counter = 0
    init_sum, style_sum = 0, 0
    for path in os.listdir(user_images_path):
        path = "".join(path.split(" "))
        img_path = user_images_path + "/" + path
        dir_name = user_images_path + "/" + path.split('.')[0]
        dir_gen = gen_images_path + "/" + path.split('.')[0]
        os.mkdir(dir_name)
        shutil.move(img_path, dir_name)
        os.mkdir(dir_gen)

        #4, 5, 6
        # пользовательская картинка должна лежать одна в директории data_path, чтобы корректно посчиталась метрика
        train_pack_path = "train_data/downloaded/adrianochelentano"
        data_convert_path = "convert_path"

        data_paths = os.listdir(train_pack_path)
        for image_path in data_paths:
            data_image = webp.load_image(train_pack_path + "/" + image_path, 'RGB')
            data_image = np.asarray(data_image)
            img = Image.fromarray(data_image, 'RGB')
            img.save(data_convert_path + "/" + image_path.split(".")[0] + ".png")

        image_path = dir_name + "/" + os.listdir(dir_name)[0]
        init_image = Image.open(image_path).convert("RGB")
        init_image = init_image.resize((256, 256))
        init_image.show()
        init_image.save(image_path)

        text = ""
        inputs = processor(init_image, text, return_tensors="pt").to("cuda")

        out = model.generate(**inputs)
        text = "a cartoon sticker of " + processor.decode(out[0], skip_special_tokens=True)
        print(text)

        gen_image = pipe(image=init_image, prompt=text, num_inference_steps=45, guidance_scale=7.5).images[0]
        gen_image.show()


        gen_image.save(dir_gen + "/generated.png")
        out_init = subprocess.getstatusoutput(f'python -m pytorch_fid {dir_name} {dir_gen} --device cuda:1')[1].split("FID:  ")[-1]
        print(out_init)

        gen_image.save(dir_gen + "/generated0.png")
        out_style = subprocess.getstatusoutput(f'python -m pytorch_fid convert_path {dir_gen} --device cuda:1')[1].split("FID:  ")[-1]
        print(out_style)

        counter += 1
        init_sum += float(out_init)
        style_sum += float(out_style)
        print(f"FID init:   {float(out_init)}\nFID style:   {float(out_style)}")

        clear_dir("gen_images_data")

    clear_dir("user_images_data")
    clear_dir("convert_path")

    print(f"Pack FID init:   {init_sum / counter}\nPack FID style:   {style_sum / counter}")
    
    final_init_sum += init_sum / counter
    final_style_sum += style_sum / counter

    
print(f"Final FID init:   {final_init_sum / len(packs_list)}\nFinal FID style:   {final_style_sum / len(packs_list)}")
'''
0. все пользовательские картинки лежат в начальной папке
1. раскидать все пользовательские картинки по одной в соответствующую папку
2. продублировать директорию с папками под будущие генерации
3. затюнить модельку на нужном стикерпаке
4. для каждой пользовательской картинки загенерить стикер
5. посчитать метрики между стикерпаком и результатами и между пользовательскими и результатами
6. профит
'''

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/tljh/user/lib/python3.9/site-packages/accelerate/accelerator.py:249: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
05/05/2023 07:48:02 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'sample_max_value', 'clip_sample_range', 'variance_type', 'thresholding', 'dynamic_thresholding_ratio'} was not found in config. Values wi

/opt/tljh/user/lib/python3.9/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening '/tmp/tmpb85bbi1o.PNG'
/opt/tljh/user/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a cartoon sticker of there is a cat that is looking up at something


...

  0%|          | 0/36 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening '/tmp/tmpkr04bfv6.PNG'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
296.8311145166668
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
259.9556243141578
FID init:   296.8311145166668
FID style:   259.9556243141578
Pack FID init:   350.9820176313567
Pack FID style:   232.25457941113274
Final FID init:   369.465668794099
Final FID style:   227.48762740011915


'\n0. все пользовательские картинки лежат в начальной папке\n1. раскидать все пользовательские картинки по одной в соответствующую папку\n2. продублировать директорию с папками под будущие генерации\n3. затюнить модельку на нужном стикерпаке\n4. для каждой пользовательской картинки загенерить стикер\n5. посчитать метрики между стикерпаком и результатами и между пользовательскими и результатами\n6. профит\n'